In [5]:
import pandas as pd

# Natural Language Processing 
import nltk
from nltk import sent_tokenize, word_tokenize
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [6]:
# parse host_since, first_review, last_review and calendar_last_scraped
df = pd.read_csv('listings.csv',
                parse_dates=['first_review', 'last_review', 'host_since', 'calendar_last_scraped'])
df.head(25)

/Users/syed/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (87) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2818,https://www.airbnb.com/rooms/2818,20181206172549,2018-12-06,Quiet Garden View Room & Super Fast WiFi,Quiet Garden View Room & Super Fast WiFi,I'm renting a bedroom (room overlooking the ga...,Quiet Garden View Room & Super Fast WiFi I'm r...,none,"Indische Buurt (""Indies Neighborhood"") is a ne...",...,f,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.10
1,3209,https://www.airbnb.com/rooms/3209,20181206172549,2018-12-06,"Quiet apt near center, great view",You will love our spacious (90 m2) bright apar...,"Our apartment has lots of light, a balcony and...",You will love our spacious (90 m2) bright apar...,none,Welcome to the Spaarndammerbuurt! From the beg...,...,f,NaN,{Amsterdam},f,f,moderate,f,f,1,1.03
2,20168,https://www.airbnb.com/rooms/20168,20181206172549,2018-12-06,100%Centre-Studio 1 Private Floor/Bathroom,"Cozy studio on your own private floor, 100% in...",For those who like all facets of city life. In...,"Cozy studio on your own private floor, 100% in...",none,Located just in between famous central canals....,...,f,NaN,{Amsterdam},f,f,strict_14_with_grace_period,f,f,2,2.18
3,25428,https://www.airbnb.com/rooms/25428,20181206172549,2018-12-06,Lovely apt in City Centre (Jordaan),NaN,"This nicely furnished, newly renovated apt is...","This nicely furnished, newly renovated apt is...",none,NaN,...,f,NaN,{Amsterdam},f,f,strict_14_with_grace_period,f,f,2,0.09
4,27886,https://www.airbnb.com/rooms/27886,20181206172549,2018-12-06,"Romantic, stylish B&B houseboat in canal district",Stylish and romantic houseboat on fantastic hi...,For a romantic couple: A beautifully restored ...,Stylish and romantic houseboat on fantastic hi...,none,"Central, quiet, safe, clean and beautiful.",...,f,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.03
5,28658,https://www.airbnb.com/rooms/28658,20181206172549,2018-12-06,Cosy guest room near city centre -1,2 beds guest room in Amsterdam West near Erasm...,Cosy small but comfortable guest room with twi...,2 beds guest room in Amsterdam West near Erasm...,none,"The listing is located in Amsterdam west, betw...",...,f,NaN,{Amsterdam},f,f,moderate,t,t,2,4.16
6,28871,https://www.airbnb.com/rooms/28871,20181206172549,2018-12-06,Comfortable double room,NaN,In a monumental house right in the center of A...,In a monumental house right in the center of A...,none,NaN,...,f,NaN,{Amsterdam},f,f,moderate,f,f,3,2.13
7,29051,https://www.airbnb.com/rooms/29051,20181206172549,2018-12-06,Comfortable single room,because of the city imposing a 4 paying guest ...,In a monumental house right in the center of A...,because of the city imposing a 4 paying guest ...,none,the street is quite lively especially on weeke...,...,f,NaN,{Amsterdam},f,f,moderate,f,f,3,4.07
8,31080,https://www.airbnb.com/rooms/31080,20181206172549,2018-12-06,2-story apartment + rooftop terrace,NaN,My apartment is light and cosy. The three bed...,My apartment is light and cosy. The three bed...,none,NaN,...,f,NaN,{Amsterdam},f,f,moderate,f,f,1,0.36
9,41125,https://www.airbnb.com/rooms/41125,20181206172549,2018-12-06,Amsterdam Center Entire Apartment,NaN,"A nice, sunny and spacious apartment in the ce...","A nice, sunny and spacious apartment in the ce...",none,NaN,...,f,NaN,{Amsterdam},f,f,moderate,f,f,1,0.78


In [8]:
# percentage of values that are missing
total_nan = df.isna().sum().sort_values(ascending=False)
percentage_nan = (total_nan / df.shape[0]) * 100
tabel = pd.concat([total_nan, percentage_nan], axis=1, keys=['Total NaN values', 'Percentage of NaN values']).head(60)
tabel

,Total NaN values,Percentage of NaN values
thumbnail_url,20030,100.000000
medium_url,20030,100.000000
host_acceptance_rate,20030,100.000000
xl_picture_url,20030,100.000000
neighbourhood_group_cleansed,20030,100.000000
license,20021,99.955067
square_feet,19624,97.973040
monthly_price,18469,92.206690
weekly_price,17187,85.806291
notes,10999,54.912631


In [9]:
# lets drop what we can and then further look into the features 
# making a list of all the columns to be dropped 
drop_cols = ['summary', 'space', 'name', 'state', 'country', 'country_code', 'review_scores_cleanliness', 'review_scores_accuracy',
            'listing_url', 'scrape_id', 'last_scraped', 'experiences_offered', 'notes', 'transit', 'access', 'interaction', 'house_rules', 
            'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_url', 'host_about', 'host_thumbnail_url', 'host_picture_url',
            'host_verifications', 'monthly_price', 'weekly_price', 'host_total_listings_count', 'calendar_updated', 'square_feet', 'host_identity_verified',
            'host_has_profile_pic']
# making a new dataframe 
airbnb = df.drop(drop_cols, axis=1)
airbnb.shape 

(20030, 63)

In [14]:
index = airbnb.index
len(index)
for i in range(len(index)):
    airbnb['price'][i] = airbnb['price'][i].replace("$","")

airbnb['price']

/var/folders/l3/x7cr88nd2lzgbz429xzz3qnh0000gn/T/ipykernel_16839/268568548.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airbnb['price'][i] = airbnb['price'][i].replace("$","")


0         59.00
1        160.00
2         80.00
3        125.00
4        150.00
          ...  
20025    340.00
20026    150.00
20027     80.00
20028     55.00
20029    220.00
Name: price, Length: 20030, dtype: object

In [15]:
# Extracting valuable information from amenities

# first step before we feed the input to CountVectorizer is to clean the text 

# Instantiate (I will be using lemmatizer but we could have also used stemmer)
lemmatizer = WordNetLemmatizer()
# empty list which will append after evry loop 
corpus_am = []
# writing a for loop to itterate over every description 
for i in range(len(airbnb)):
    # remove evrerything but wordsa sp starting from a to z
    review = re.sub('[^a-zA-Z]', ' ', airbnb.amenities[i])
    # to change everything to lower case
    review = review.lower()
    # to split the sentence 
    review = review.split()
    # for every word in review, lemmatize the word if it is not in stopwords
    #review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words('english')) ]
    
    review = ' '.join(review)
    # append the words in the empty list created above
    corpus_am.append(review)
    

In [16]:
# # creating the bag of words model 

#passing the list of amenities that i am interested in
vect = CountVectorizer(analyzer='word', ngram_range=(1, 2), vocabulary=['breakfast',  'cable tv', 'elevator', 'free parking', 'internet', 'kitchen', 'wifi', 'family','kid friendly', 'smoke detector', 'patio' ,'balcony', 'garden','backyard', 'air conditioning', 'pets allowed', 'pool', 'coffee maker'])



# fit corpus_am
para_dtm = vect.fit(corpus_am)

# examine the fitted vocabulary
vect.get_feature_names()

/Users/syed/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['breakfast',
 'cable tv',
 'elevator',
 'free parking',
 'internet',
 'kitchen',
 'wifi',
 'family',
 'kid friendly',
 'smoke detector',
 'patio',
 'balcony',
 'garden',
 'backyard',
 'air conditioning',
 'pets allowed',
 'pool',
 'coffee maker']

In [17]:
# now we will transform the corpus_am into a 'document-term matrix'
# since this outputs a sparse matrix, we will convert in into an array so we could use that to make our airbnb dataframe
amenities_dtm = vect.transform(corpus_am).toarray()

In [18]:
# we will convert this into a dataframe so we could merge with our airbnb dataframe
amenities_dtm = pd.DataFrame(amenities_dtm, columns=vect.get_feature_names())
amenities_dtm

,breakfast,cable tv,elevator,free parking,internet,kitchen,wifi,family,kid friendly,smoke detector,patio,balcony,garden,backyard,air conditioning,pets allowed,pool,coffee maker
0,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,0,0,0
1,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,1
2,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
3,0,1,1,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20025,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0
20026,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
20027,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
20028,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0


In [19]:
amenities_dtm = amenities_dtm.join(airbnb['price'], lsuffix='_caller', rsuffix='_other')

In [20]:
amenities_dtm = amenities_dtm.join(airbnb['id'], lsuffix='_caller', rsuffix='_other')

In [21]:
amenities_dtm.to_csv(r'Amineties.csv', index = False)